# 🎯 RNA-seqデータ解析
## 📚 手順概要

1. データ読み込み（TPMデータ）
2. 階層的クラスタリング
3. ボルケーノプロット
4. 遺伝子リストの出力

## 📥 1. データ読み込み（TPMデータ）
関連ライブラリを読み込む

In [ ]:
# データ操作や解析に使う Python ライブラリ「pandas」を pd という名前で使えるようにするための宣言
import pandas as pd

# 数値計算ライブラリ NumPy を np という名前で使えるようにする
import numpy as np

TPMデータをファイルから読み込む。

In [ ]:
df_tpm = pd.read_excel("https://github.com/iwata97/bioinfo/raw/refs/heads/main/TPM_data.xlsx")

中身を確認してみる

In [ ]:
df_tpm.head()

## 🌐 2. 階層的クラスタリング

データの準備<br>
gene列を削除し、値はlog2に変換する

In [ ]:
tpm = df_tpm.drop('gene', axis=1)
log_tpm = np.log2(tpm + 1.0)

行と列を入れ替える（転置）。階層的クラスタリングをする際に、行と列を入れ替えておくとやりやすいため。

In [ ]:
# 転置
tpm_t = tpm.T
tpm_t

データセットは準備できた。<br>
このデータを用いて、batch_1, 2, 3とchemostat_1, 2, 3の6個を階層的クラスタリングを実施する。<br>
階層的クラスタリングとは（参照：https://datawokagaku.com/hierarchical_clustering/ ）

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

linkage_result = linkage(tpm_t, method='ward', metric='euclidean')
plt.figure(num=None, figsize=(16, 9), dpi=200, facecolor='w', edgecolor='k')
dendrogram(linkage_result, labels=list(tpm_t.index))
plt.show()

## 🔬 3. ボルケーノプロット

関連ライブラリを読み込む

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

tpm DataFrame に 6サンプル（前半3つが batch、後半3つが chemostat）あるので、それぞれをlog_batchとlog_chemoに入れる。

In [ ]:
log_batch = log_tpm.iloc[:, :3]
log_chemo = log_tpm.iloc[:, 3:]

各遺伝子について Welch のt検定を行い、fold changeとp値を計算

In [ ]:
# 各遺伝子について Welch のt検定を行い、fold changeとp値を計算
log2fc = log_chemo.mean(axis=1) - log_batch.mean(axis=1)
pvals = []
for i in range(len(tpm)):
    stat, pval = ttest_ind(log_batch.iloc[i], log_chemo.iloc[i], equal_var=False)
    pvals.append(pval)

pvals = np.array(pvals)
neg_log10_pvals = -np.log10(pvals)

Differentially Expressed Genes（DEG）を定義する。<br>
（fold change > 2 かつ p < 0.05）とする。

In [ ]:
# DEGフラグ（fold change > 2 かつ p < 0.05）
is_DEG = (np.abs(log2fc) > 2) & (pvals < 0.05)

ボルケーノプロットを描く。<br>

In [ ]:
# --- ボルケーノプロット ---
fig, ax = plt.subplots(figsize=(10, 6))
ax.grid()

# 非DEG（灰色）
ax.scatter(log2fc[~is_DEG], neg_log10_pvals[~is_DEG],
           color='gray', alpha=0.3, label='non-DEG')

# DEG（赤）
ax.scatter(log2fc[is_DEG], neg_log10_pvals[is_DEG],
           color='red', alpha=0.6, label='DEG')

# 軸としきい値ライン
ax.axhline(-np.log10(0.05), color='blue', linestyle='--', linewidth=1)
ax.axvline(2, color='blue', linestyle='--', linewidth=1)
ax.axvline(-2, color='blue', linestyle='--', linewidth=1)

ax.set_xlabel('log2 fold change (chemostat - batch)')
ax.set_ylabel('-log10(p-value)')
ax.set_title('Volcano Plot')
ax.legend()
plt.show()


## 練習
DEGの定義を変えてボルケーノプロットを描いてみましょう。
例えば、<br>
・fold change > 3 かつ p < 0.05<br>
・fold change > 2 かつ p < 0.01

## 📥 4. 遺伝子リストの出力

次に、DEGの定義に当てはまる遺伝子のリストを出力してみる。<br>
リストは、p値でソートしておく。

In [ ]:
# gene列を使って、元のデータに計算結果を統合
results_df = df_tpm[['gene']].copy()
results_df['log2FC'] = log2fc
results_df['pval'] = pvals
results_df['-log10(pval)'] = -np.log10(pvals)
results_df['is_DEG'] = is_DEG

# DEGの行だけ抽出して、p値でソート
deg_df = results_df[results_df['is_DEG']].sort_values(by='pval')

# 上位10件だけ表示（必要に応じて全件表示も可能）
print(deg_df.head(10))

# CSVに出力する場合（Google Colabならこのファイルがダウンロード可能）
deg_df.to_csv("DEG_list_sorted_by_pvalue.csv", index=False)
